In [49]:
import sys
sys.path.append('./..')
from models.region_loss import Regional_Loss
from scripts import load_dataset
import torch
import glob
import pandas as pd
from torch.utils.tensorboard import SummaryWriter
from create_datasets_from_embddings import create_datasets_from_embddings
from models import nn




In [50]:
def add_metrics_and_plot_tb(criterion,writer, outputs, targets, name, step, training_dataset_name, seed, starting_regional_loss_portion, regional_loss_decline, timestamp):
        per_class_precision, per_class_recall, per_class_f1, _, target_idx = criterion.calculate_metrics_per_class(outputs, targets)
        writer.add_scalar(f'{name} avg Class Precision', per_class_precision.mean(), step)
        writer.add_scalar(f'{name} avg Class Recall', per_class_recall.mean(), step)
        writer.add_scalar(f'{name} avg Class F1', per_clwriter = SummaryWriter(
            log_dir=f'runs/seed_{seed}/{training_dataset_name[:-4]}/starting_regional_loss_portion-{starting_regional_loss_portion}/regional_loss_decline-{regional_loss_decline}/{timestamp}')ass_f1.mean(), step)

        metrics_df = pd.DataFrame({'Precision': per_class_precision, 'Recall': per_class_recall, 'Fscore': per_class_f1})
        metrics_df.index = target_idx
        
        ignored_classes = metrics_df[(metrics_df['Precision'] == 0) & (metrics_df['Recall'] == 0)]
        metrics_df = metrics_df.drop(ignored_classes.index)

        bar_plot = metrics_df.plot(kind='bar', xlabel='Class', ylabel='Metrics', title='Metrics per Country, {len(ignored_classes)} ignored countries.').get_figure()
        writer.add_figure(f'{name} Metrics per Country', bar_plot, step)
        writer.add_scalar(f'{name} Number of Ignored Classes', len(ignored_classes), step)

        # Calculate metrics per region
        per_region_precision, per_region_recall, per_region_f1, _, region_index = criterion.calculate_metrics_per_region(outputs, targets)
        writer.add_scalar(f'{name} avg Region Precision', per_region_precision.mean(), step)
        writer.add_scalar(f'{name} avg Region Recall', per_region_recall.mean(), step)
        writer.add_scalar(f'{name} avg Region F1', per_region_f1.mean(), step)
        region_metrics_df = pd.DataFrame({'Precision': per_region_precision, 'Recall': per_region_recall, 'Fscore': per_region_f1})


        region_metrics_df.index = region_index
        
        ignored_regions = region_metrics_df[(region_metrics_df['Precision'] == 0) & (region_metrics_df['Recall'] == 0)]
        region_metrics_df = region_metrics_df.drop(ignored_regions.index)

        region_bar_plot = region_metrics_df.plot(kind='bar', xlabel='Region', ylabel='Metrics', title=f'Metrics per Region, {len(ignored_regions)} ignored regions.').get_figure()
        writer.add_figure(f'{name} Metrics per Region', region_bar_plot, step)
        writer.add_scalar(f'{name} Number of Ignored Regions', len(ignored_regions), step)
        writer.add_text(f'{name} List of Ignored Regions', ';'.join(ignored_regions.index.to_list()), step)
        writer.flush()

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3634239525.py, line 5)

In [ ]:
seeds = [4808,4947,5723,3838,5836,3947,8956,5402,1215,8980]
training_datasets = [
        'geo_weakly_balanced',
        'geo_unbalanced',
        'geo_strongly_balanced',
        'mixed_weakly_balanced',
        'mixed_strongly_balanced'
    ]
hyperparameters = [
            'starting_regional_loss_portion-0.0',
            'starting_regional_loss_portion-0.25',
            'starting_regional_loss_portion-0.8',
            'starting_regional_loss_portion-0.5',
        ]
saved_modesl_path = "/home/lbrenig/Documents/Uni/GPML/good_practices_ml/saved_models/"

In [51]:

REPO_PATH = '/home/lbrenig/Documents/Uni/GPML/good_practices_ml'
country_list = f'{REPO_PATH}/country_list/country_list_region.csv'
region_list = f'{REPO_PATH}/country_list/UNSD_Methodology.csv'



testing_directory = f'{REPO_PATH}/Embeddings/Testing'

model_paths = glob.glob('/home/lbrenig/Documents/Uni/GPML/good_practices_ml/saved_models/*epoch_15*')
model_paths = [path for path in model_paths if int(path.split("_")[-4][-2:]) > 7]
model_paths = sorted(model_paths, key=lambda path: int(''.join(path.split("_")[-4:-3])))

counter = 0
country_list = pd.read_csv(country_list)
criterion = Regional_Loss(country_list)
for seed in seeds:
    #reate_datasets_from_embddings(seed=seed,REPO_PATH=REPO_PATH)
    test_df = pd.read_csv(f'{testing_directory}/knwon_test_data.csv')
    test_dataset = load_dataset.EmbeddingDataset_from_df(
    test_df, "test")
    for dataset in training_datasets:
        for hyperparameter in hyperparameters:
            path = model_paths[counter]
            model = nn.FinetunedClip()
            model.load_state_dict(torch.load(path))
            model.eval()
            torch.manual_seed(seed)
            writer = SummaryWriter(
            log_dir=f'runs/test_experiment1/seed_{seed}/{dataset}/starting_regional_loss_portion-{hyperparameter}/regional_loss_decline-/{1515}')
            inputs, targets = test_dataset[:]
            outputs = model(inputs)
            
            avg_test_region_accuracy = criterion.claculate_region_accuracy(
                outputs, targets)
            avg_test_accuracy = criterion.calculate_country_accuracy(
                outputs, targets)
            
            writer.add_scalar(
                'Test Accuracy', avg_test_accuracy, )
            writer.add_scalar('Test Regional Accuracy',
                                avg_test_region_accuracy)

            with torch.no_grad():
                predicitions_idx = torch.argmax(outputs, axis=1).tolist()
                target_idx = [country_list[country_list['Country'] == target].index[0] for target in targets]
                
                try:
                    add_metrics_and_plot_tb(criterion,writer, outputs, targets, 'Test', 0, dataset, seed, hyperparameter, 0, '1515')
                except Exception as e:
                    print(e)


TypeError: Expected state_dict to be dict-like, got <class 'method'>.